In [59]:
import json
import cobrakbase
import escher
import modelseed_escher
from cobrakbase.core import KBaseFBAModel
from cobrakbase.core import KBaseGenome
import pandas as pd

In [2]:
token = "64XQ7SABQILQWSEW3CQKZXJA63DXZBGH"
kbase = cobrakbase.KBaseAPI(token)

In [6]:
escher_seed = modelseed_escher.EscherManager(escher)

In [4]:
ws = 'filipeliu:narrative_1582914694010'
model1 = kbase.get_object('GCF_000005845.2.RAST2.mdl_v3', ws)
model2 = kbase.get_object('GCF_000005845.2.RASTtk.mdl', ws)

In [9]:
models = {
    'curated' : KBaseFBAModel(model1),
    'old' : KBaseFBAModel(model2),
}

In [13]:
%run ../lib/kb_escher/kb_escher_utils.py

In [14]:
map_list = {}
for m in escher_seed.list_maps('ModelSEED'):
    model_id, map_id = m.split('.')
    map_data = escher_seed.get_map('ModelSEED', model_id, map_id)
    map_list[map_id] = {}
    for model_id in models:
        map_list[map_id][model_id] = {}
        map_list[map_id][model_id] = scan_content(map_data, models[model_id], None)
with open('../data/html/data/map_list.json', 'w') as fh:
    fh.write(json.dumps(map_list))

In [15]:
fbamodel = models['curated']

In [29]:
for model_id in models:
    fbamodel = models[model_id]
    genes = set()
    for rxn in fbamodel.reactions:
        for and_rule in rxn.get_gpr():
            genes |= and_rule
    common_set &= genes

In [27]:
common_set = set()
common_set |= genes

In [30]:
len(common_set)

838

In [40]:
model_unique = {}
for model_id in models:
    unique = set()
    fbamodel = models[model_id]
    genes = set()
    for rxn in fbamodel.reactions:
        for and_rule in rxn.get_gpr():
            genes |= and_rule
    unique = genes - common_set
    model_unique[model_id] = unique

In [39]:
model_genomes = {}
for model_id in models:
    fbamodel = models[model_id]
    ref = kbase.get_object_info_from_ref(fbamodel.data['genome_ref'])
    o = kbase.get_object(ref.id, ref.workspace_id)
    model_genomes[model_id] = KBaseGenome(o)

In [55]:
data = {
    'model_unique' : [],
    'gene_id' : [],
    'gene_annotation1' : [],
    'gene_annotation2' : [],
}
for model_id in models:
    for gene_id in model_unique[model_id]:
        data['model_unique'].append(model_id)
        data['gene_id'].append(gene_id)
        genome1 = model_genomes['curated']
        genome2 = model_genomes['old']
        f1 = list(filter(lambda x: x['id'] == gene_id, genome1.features))[0]
        f2 = list(filter(lambda x: x['id'] == gene_id, genome2.features))[0]
        data['gene_annotation1'].append(genome1.get_annotation2f(f1))
        data['gene_annotation2'].append(genome2.get_annotation2f(f2))

In [57]:
df = pd.DataFrame(data)

In [60]:
kbase.ws_client.get_workspace_info({
    'id' : '56739'
})

[56739,
 'filipeliu:narrative_1582914694010',
 'filipeliu',
 '2020-03-02T19:22:50+0000',
 26,
 'a',
 'n',
 'unlocked',
 {'cell_count': '1',
  'narrative_nice_name': 'Updated Template',
  'searchtags': 'narrative',
  'is_temporary': 'false',
  'narrative': '1'}]

In [58]:
df.to_csv('/Users/fliu/workspace/jupyter/data/modelseed2/ecoli_rast_comp.tsv', sep='\t')